In [1]:
import numpy as onp
import tensorflow as tf
import matplotlib.pyplot as plt

from utils import *

In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only use the first GPU
    try:
        tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
    except RuntimeError as e:
    # Visible devices must be set before GPUs have been initialized
        print(e)

1 Physical GPUs, 1 Logical GPU


# data

In [3]:
DATASET = 'imagenet'
class_num   = 2
image_shape = (224, 224, 3)

batch_size = 32
train_size = None
test_size = 100

epochs = 50
eps = 0.03

In [4]:
def one_hot(x, k, dtype=np.float64):
    """Create a one-hot encoding of x of size k."""
    return onp.array(x[:, None] == onp.arange(k), dtype)

In [5]:
x_train = onp.load('../../Jimmy/ntk_nngp/source/dataset/imagenet_x_train.npy')[:2000]
y_train = onp.load('../../Jimmy/ntk_nngp/source/dataset/imagenet_y_train.npy')[:2000]

x_valid = onp.load('../../Jimmy/ntk_nngp/source/dataset/imagenet_x_train.npy')[2000:]
y_valid = onp.load('../../Jimmy/ntk_nngp/source/dataset/imagenet_y_train.npy')[2000:]

x_test = onp.load('../../Jimmy/ntk_nngp/source/dataset/imagenet_x_test.npy')
y_test = onp.load('../../Jimmy/ntk_nngp/source/dataset/imagenet_y_test.npy')

In [6]:
# shuffle
seed = 0
x_train, y_train = shaffle(x_train, y_train, seed)

In [7]:
train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_ds = train_ds.shuffle(
    5000
).batch(
    batch_size
).prefetch(10)

In [8]:
valid_ds = tf.data.Dataset.from_tensor_slices((x_valid, y_valid))
valid_ds = valid_ds.batch(20).prefetch(10)

# model

In [9]:
layers = tf.keras.layers

In [10]:
img_input = layers.Input(shape=image_shape)
x = layers.Conv2D(64, (3, 3), activation='relu', padding='same', 
                  kernel_initializer=tf.keras.initializers.GlorotNormal())(img_input)
x = layers.Conv2D(64, (3, 3), activation='relu', padding='same',
                 kernel_initializer=tf.keras.initializers.GlorotNormal())(x)
x = layers.MaxPooling2D()(x)
x = layers.Conv2D(128, (3, 3), activation='relu', padding='same', 
                  kernel_initializer=tf.keras.initializers.GlorotNormal())(x)
x = layers.Conv2D(128, (3, 3), activation='relu', padding='same',
                 kernel_initializer=tf.keras.initializers.GlorotNormal())(x)
x = layers.MaxPooling2D()(x)
x = layers.Conv2D(256, (3, 3), activation='relu', padding='same', 
                  kernel_initializer=tf.keras.initializers.GlorotNormal())(x)
x = layers.Conv2D(256, (3, 3), activation='relu', padding='same',
                 kernel_initializer=tf.keras.initializers.GlorotNormal())(x)
x = layers.Conv2D(256, (3, 3), activation='relu', padding='same',
                 kernel_initializer=tf.keras.initializers.GlorotNormal())(x)
x = layers.MaxPooling2D()(x)
x = layers.Conv2D(512, (3, 3), activation='relu', padding='same', 
                  kernel_initializer=tf.keras.initializers.GlorotNormal())(x)
x = layers.Conv2D(512, (3, 3), activation='relu', padding='same',
                 kernel_initializer=tf.keras.initializers.GlorotNormal())(x)
x = layers.Conv2D(512, (3, 3), activation='relu', padding='same',
                 kernel_initializer=tf.keras.initializers.GlorotNormal())(x)
x = layers.MaxPooling2D()(x)
x = layers.Conv2D(512, (3, 3), activation='relu', padding='same', 
                  kernel_initializer=tf.keras.initializers.GlorotNormal())(x)
x = layers.Conv2D(512, (3, 3), activation='relu', padding='same',
                 kernel_initializer=tf.keras.initializers.GlorotNormal())(x)
x = layers.Conv2D(512, (3, 3), activation='relu', padding='same',
                 kernel_initializer=tf.keras.initializers.GlorotNormal())(x)
x = layers.MaxPooling2D()(x)
x = layers.Flatten()(x)
x = layers.Dense(512, activation='relu', kernel_initializer=tf.keras.initializers.GlorotNormal())(x)
out = layers.Dense(class_num, kernel_initializer=tf.keras.initializers.GlorotNormal())(x)

model = tf.keras.Model(inputs=img_input, outputs=out)

In [11]:
def scheduler(epoch, lr):
    if epoch < 10:
        return 1e-3
    elif epoch < 90:
        return 1e-2
    elif epoch < 150:
        return 1e-3
    else:
        return 1e-4

In [12]:
model.compile(optimizer=tf.keras.optimizers.SGD(momentum=0.5),
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [13]:
callback = tf.keras.callbacks.LearningRateScheduler(scheduler)

In [14]:
model.fit(x=train_ds, validation_data=valid_ds, epochs=epochs, callbacks=[callback])

Epoch 1/50
63/63 [==============================] - 14s 221ms/step - loss: 0.6911 - accuracy: 0.5550 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/50
63/63 [==============================] - 8s 121ms/step - loss: 0.6885 - accuracy: 0.5550 - val_loss: 0.7237 - val_accuracy: 0.0000e+00
Epoch 3/50
63/63 [==============================] - 5s 75ms/step - loss: 0.6868 - accuracy: 0.5550 - val_loss: 0.7340 - val_accuracy: 0.0000e+00
Epoch 4/50
63/63 [==============================] - 5s 74ms/step - loss: 0.6845 - accuracy: 0.5550 - val_loss: 0.7438 - val_accuracy: 0.0000e+00
Epoch 5/50
63/63 [==============================] - 5s 72ms/step - loss: 0.6820 - accuracy: 0.5550 - val_loss: 0.7533 - val_accuracy: 0.0000e+00
Epoch 6/50
63/63 [==============================] - 5s 72ms/step - loss: 0.6789 - accuracy: 0.5550 - val_loss: 0.7585 - val_accuracy: 0.0000e+00
Epoch 7/50
63/63 [==============================] - 5s 72ms/step - loss: 0.6742 - accuracy: 0.5550 - val_loss: 0.7628 - val

In [15]:
model.save_weights('./model_weights/vgg16_train=imagenet_all-without-DA_ce.h5')

In [16]:
model.evaluate(x=x_test, y=y_test, verbose=0)

[0.22463561279466376, 0.92]